In [1]:
# merge and rename
import shutil
import os
path = "/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/Defect_origin"
despath = "/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/merge"
for root,dirs,files in os.walk(path):
    for file in files:
        shutil.copy(os.path.join(root,file),os.path.join(os.path.join(despath,file)))
        os.rename(os.path.join(despath,file),os.path.join(despath,file[:-4]+'.jpg'))

In [2]:
import xml.etree.cElementTree as ET
import cv2
import os
import numpy as np

crop_path = "/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/Defect_crop"
ori_path = "/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/merge"
xml_path = "/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/result/xml"
des_path = "/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/result/picture"
        

In [ ]:
for root,dirs,files in os.walk(crop_path):
    for file in files:
        if os.path.exists(os.path.join(ori_path,file[:-4]+".jpg")):
            print(os.path.join(root,file))
            try:
                cropImg = cv2.imread(os.path.join(root,file))
                oriImg = cv2.imread(os.path.join(ori_path,file[:-4]+".jpg"))
                result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
                x_and_y = np.unravel_index(result.argmax(),result.shape)
                print(x_and_y)

                tree = ET.parse('/ncudeep/yuan/Defect_part/1222-1227/ForFRCNN/8800AAHJFAMYC108100001-01.xml')
                tree_root = tree.getroot()
                change_dict = {"filename":file,"width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                               "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split("/")[-1]}
                for key in change_dict.keys():
                    for e in tree_root.iter(key):
                        e.text = str(change_dict[key])
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
            except:
                print("continue")

In [8]:
# use xml to catch picture
for root,dirs,files in os.walk(crop_path):
    for file in files:
        if os.path.exists(os.path.join(ori_path,file[:-4]+".jpg")) and\
                          os.path.exists(os.path.join(xml_path,file[:-4]+".xml")):
            if not os.path.exists(os.path.join(des_path,root.split("/")[-1])):
                os.mkdir(os.path.join(des_path,root.split("/")[-1]))
            shutil.move(os.path.join(ori_path,file[:-4]+".jpg"),os.path.join(des_path,root.split("/")[-1],file[:-4]+".jpg"))